1. объединить в одну выборку  
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)  
3. Проверить насколько хорошо работают подходы  

In [1]:
import pandas as pd
import numpy as np
import re
import os

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk import ngrams

from collections import Counter
from nltk.tokenize import word_tokenize

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

import annoy
from gensim.models import Word2Vec, FastText
import pickle

from tqdm import tqdm_notebook

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

# объединить в одну выборку

In [2]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

# на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов (на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)

In [3]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


 функция удаления слов которые начинаются с @

In [4]:
def funk_del(input_txt):
    pattern = "@[\w]*"
    if re.findall(pattern, input_txt):
        return re.sub(pattern, ' ', input_txt)
    else:
        return re.sub(pattern, ' ', input_txt)


Очищаю твиты от слов которые начинаются с @, убираю знаки препинания и все не русские буквы, привожу к нижнему ригистру

In [5]:
df['text'] = df.text.apply(funk_del)

In [6]:
# pattern = r'[^\w\s]'
# df['text'] = df['text'].apply(lambda x: re.sub(pattern, ' ', x))

In [7]:
pattern = r'[^а-яА-Я0-9]'
df['text'] = df['text'].apply(lambda x: re.sub(pattern, ' ', x))

In [8]:
# df['text'] = df['text'].str.lower()

In [9]:
# df['text'] = df['text'].apply(lambda x: [''.join(str(i.lower())) for i in x.split()])

In [7]:
df.head()

,text,label
0,хоть я и школота но поверь у нас то же сам...,positive
1,Да все таки он немного похож на него Но мой ...,positive
2,Ну ты идиотка я испугалась за тебя,positive
3,Кто то в углу сидит и погибает от голод...,positive
4,Вот что значит страшилка Но блин посмотре...,positive


In [11]:
df.shape

(226834, 2)

функция preprocess_txt еще одна обработка текста, и еще приводит слова к нормальной форме

In [8]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

предобработка текста и приведение к нормальной форме

In [16]:
sentences = []
counter = 0
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0
for line in tqdm_notebook(df.text):
#     print(line)
    spls = preprocess_txt(line)
    sentences.append(spls)
    
    counter += 1
        
    if counter > 100000:
        break
    

  0%|          | 0/226834 [00:00<?, ?it/s]

In [17]:
# sentences = [i for i in sentences if len(i) > 2]

In [18]:
sentences[0]

['школотый', 'поверь', 'самый', 'общество', 'профилировать', 'предмет', 'тип']

создание модели word2Vec

In [19]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=2)


создание модели fasttext

In [20]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=2, window=5, workers=8)

In [21]:
# texts = [line for line in df.text]

создаю вектора длиной в 300 символов, где каждый вектор описывает один твит

In [22]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in tqdm_notebook(df.text):
        n_w2v = 0
        n_ft = 0
        spls = line
        index_map[counter] = spls
        question = preprocess_txt(spls)
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        # ограничение в  100000 строк
        if counter > 100000:
            break
    

w2v_index.build(10)
ft_index.build(10)

  0%|          | 0/226834 [00:00<?, ?it/s]

True

функция поиска ближайших 5 твитов

In [75]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
#     print(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [31]:
TEXT = df.text.iloc[1]
# TEXT = 'покажи мне коментарии'
TEXT

'Да  все таки он немного похож на него  Но мой мальчик все равно лучше  '

In [41]:
'да  все таки он немного похож на него  но мой мальчик все равно лучше  ' in a

True

5 похожих твитов на можели w2w

In [76]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['да  все таки он немного похож на него  но мой мальчик все равно лучше  ',
 'все равно  я бы чувствовала себя лучше  если бы здесь была женщина детектив                  ',
 'поступки  но если материшься  выражаешься  то все равно человек не воспитанный всего хорошего    ',
 'ребятки  вы все равно молодцы  хоть один раз проиграли  для нас вы лучшие  ',
 'у меня      в папке                       таки вирус поймал  какой хороший не эмулятор    ']

5 похожих твитов на можели fasttext

In [35]:
get_response(TEXT, ft_index, modelFT, index_map)

['да  все таки он немного похож на него  но мой мальчик все равно лучше  ',
 '    если честно  так все красавчики      и плохие и хорошие ',
 '  неожиданно но красиво  все равно вы лучше исполняете песню              ',
 'все равно  я бы чувствовала себя лучше  если бы здесь была женщина детектив                  ',
 'и вс  таки  дпсники хорошие люди   второй раз уже фортит ']

# embedding

In [114]:
df.text = df.text.str.lower()

In [115]:
df.head()

,text,label
0,хоть я и школота но поверь у нас то же сам...,positive
1,да все таки он немного похож на него но мой ...,positive
2,ну ты идиотка я испугалась за тебя,positive
3,кто то в углу сидит и погибает от голод...,positive
4,вот что значит страшилка но блин посмотре...,positive


токенизация твитов

In [8]:
df['token'] = df.text.apply(lambda x: x.split())

добавляю в список все слова из каждого твита

In [9]:
words = []
for i in df.token:
    for j in i:
#         print(j)
        words.append(j)
#     print(i[0])

делаю список уникальных слов

In [10]:
print(len(words))
print(len(set(words)))
words = set(words)

2590362
204774


словарь где ключ слово, а значение индекс

In [11]:
dict_w = {}
dict_w = {v: i for i, v in enumerate(words)}

In [12]:
df.head()

,text,label,token
0,хоть я и школота но поверь у нас то же сам...,positive,"[хоть, я, и, школота, но, поверь, у, нас, то, ..."
1,Да все таки он немного похож на него Но мой ...,positive,"[Да, все, таки, он, немного, похож, на, него, ..."
2,Ну ты идиотка я испугалась за тебя,positive,"[Ну, ты, идиотка, я, испугалась, за, тебя]"
3,Кто то в углу сидит и погибает от голод...,positive,"[Кто, то, в, углу, сидит, и, погибает, от, гол..."
4,Вот что значит страшилка Но блин посмотре...,positive,"[Вот, что, значит, страшилка, Но, блин, посмот..."


функция для добавления индексов слов в df

In [13]:
def f(line):
    r = []
    for i in line:
        r.append(dict_w[i])
    return r
        

добавляю столбец с индексами

In [14]:
df['vek'] = df.token.apply(f)


In [15]:
# df['vek'] = df.vek.apply(lambda x: np.array(list(x)))

1- позитивный коментарий, 0- отрицательный, новый столбец с лейблами

In [16]:
df['lab_vek'] = [1 if i == 'positive' else 0  for i in df.label]

In [17]:
# df['vek'] = df['vek'].apply(lambda x: str(x)[1:-1])

In [18]:
# df['test'] = df['vek'].str[0]

In [19]:
# df['test'] = df['test'].apply(lambda x: str(x))

In [20]:
df.head()

,text,label,token,vek,lab_vek
0,хоть я и школота но поверь у нас то же сам...,positive,"[хоть, я, и, школота, но, поверь, у, нас, то, ...","[31599, 188570, 166109, 31966, 187390, 151384,...",1
1,Да все таки он немного похож на него Но мой ...,positive,"[Да, все, таки, он, немного, похож, на, него, ...","[97274, 147672, 115955, 30843, 72966, 30157, 1...",1
2,Ну ты идиотка я испугалась за тебя,positive,"[Ну, ты, идиотка, я, испугалась, за, тебя]","[164324, 50357, 81498, 188570, 117783, 159629,...",1
3,Кто то в углу сидит и погибает от голод...,positive,"[Кто, то, в, углу, сидит, и, погибает, от, гол...","[114942, 114232, 102995, 169357, 192986, 16610...",1
4,Вот что значит страшилка Но блин посмотре...,positive,"[Вот, что, значит, страшилка, Но, блин, посмот...","[127992, 113185, 116010, 55841, 840, 26103, 72...",1


вычисляю длину самого большого ввектора

In [21]:
a = 0
for i in df.vek:
    if len(i) > a:
        a = len(i)
print(a)

34


In [22]:
maxlen = 34

добиваю все векторы до длины 34, те которые меньше добавляются нули

In [25]:
import tensorflow as tf

In [28]:
from tensorflow.keras.utils import pad_sequences

In [29]:
df['vek_one_size'] = pad_sequences(df.vek, maxlen=maxlen, padding='post', value=0).tolist()

In [30]:
df.head()

,text,label,token,vek,lab_vek,vek_one_size
0,хоть я и школота но поверь у нас то же сам...,positive,"[хоть, я, и, школота, но, поверь, у, нас, то, ...","[31599, 188570, 166109, 31966, 187390, 151384,...",1,"[31599, 188570, 166109, 31966, 187390, 151384,..."
1,Да все таки он немного похож на него Но мой ...,positive,"[Да, все, таки, он, немного, похож, на, него, ...","[97274, 147672, 115955, 30843, 72966, 30157, 1...",1,"[97274, 147672, 115955, 30843, 72966, 30157, 1..."
2,Ну ты идиотка я испугалась за тебя,positive,"[Ну, ты, идиотка, я, испугалась, за, тебя]","[164324, 50357, 81498, 188570, 117783, 159629,...",1,"[164324, 50357, 81498, 188570, 117783, 159629,..."
3,Кто то в углу сидит и погибает от голод...,positive,"[Кто, то, в, углу, сидит, и, погибает, от, гол...","[114942, 114232, 102995, 169357, 192986, 16610...",1,"[114942, 114232, 102995, 169357, 192986, 16610..."
4,Вот что значит страшилка Но блин посмотре...,positive,"[Вот, что, значит, страшилка, Но, блин, посмот...","[127992, 113185, 116010, 55841, 840, 26103, 72...",1,"[127992, 113185, 116010, 55841, 840, 26103, 72..."


создаю слой эмбединга

In [31]:
embedding_layer = tf.keras.layers.Embedding(len(words), 7)

словарь с ембедингами твитов, где ключи соответствуют индексам в df

In [59]:
def get_emb(df):
    dict_emb = {}
    count = 0
    for i in df:
#         print(embedding_layer(tf.constant(i)))
        dict_emb[count] = embedding_layer(tf.constant(i))
        count += 1
    return dict_emb

In [60]:
dict_emb = get_emb(df.vek_one_size)

embedding первого твита

In [63]:
dict_emb[0]

<tf.Tensor: shape=(34, 7), dtype=float32, numpy=
array([[ 0.00977034, -0.03455224,  0.04629418,  0.03052551,  0.02587575,
        -0.01070384, -0.01017529],
       [ 0.03345705, -0.0029008 , -0.02111567, -0.02571675,  0.01195462,
        -0.00502092,  0.04919385],
       [-0.02503277,  0.03344201, -0.04725479, -0.00990544, -0.03153793,
        -0.01162053, -0.03319197],
       [ 0.0321652 ,  0.02268106, -0.00964663, -0.01060442, -0.02739328,
        -0.0037198 ,  0.02892692],
       [-0.02828871,  0.03760097, -0.03324286, -0.02010853,  0.02569899,
        -0.01074324,  0.00207061],
       [ 0.03095045,  0.02113045,  0.02160213, -0.01460637,  0.01193698,
         0.04087843,  0.0350023 ],
       [-0.03809605,  0.0353162 ,  0.0184479 ,  0.03700775, -0.00332605,
        -0.03477263,  0.03660217],
       [-0.04424867, -0.00620357,  0.04726447,  0.03852836,  0.02127187,
         0.00116513,  0.04559125],
       [-0.01349796,  0.00892706,  0.03161485, -0.01996101, -0.03489142,
         0.026